<a href="https://colab.research.google.com/github/KAFE45/Colab-LabBCI/blob/main/Can_You_Help_Me_Pls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyxdf
!pip install mne

In [ ]:
import pyxdf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft
from scipy.signal import butter, filtfilt
from google.colab import files
import io

# 1. อัปโหลดไฟล์
uploaded = files.upload()
file_name = list(uploaded.keys())[0]  # เลือกไฟล์แรกที่อัปโหลด

# ฟังก์ชันในการโหลดข้อมูลจากไฟล์ XDF
def load_data(filepath):
    streams, header = pyxdf.load_xdf(filepath)

    eeg_stream = None
    marker_stream = None
    for stream in streams:
        if stream['info']['name'][0] == 'eeg' and stream['info']['type'][0] == 'signal':
            eeg_stream = stream
        elif stream['info']['type'][0] == 'Markers':
            marker_stream = stream

    if eeg_stream is None:
        raise ValueError(f"ไม่พบ stream ชื่อ 'eeg' ในไฟล์ {filepath}")

    data = eeg_stream['time_series']
    sampling_rate = float(eeg_stream['info']['nominal_srate'][0])
    channel_names = eeg_stream['info']['desc'][0]['channels'][0]['channel']
    channel_labels = [chan['label'][0] for chan in channel_names]

    df = pd.DataFrame(data, columns=channel_labels)

    markers = []
    if marker_stream:
        marker_times = marker_stream['time_stamps']
        marker_values = marker_stream['time_series']
        markers = list(zip(marker_times, marker_values))

    return df, sampling_rate, markers

In [ ]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import mne
from google.colab import files
import io
import pyxdf

# Load data from the XDF file
streams, header = pyxdf.load_xdf('/content/sub-DataSSVEP551_ses-S002_task-Default_run-001_eeg (1).xdf')

# Assume eeg_stream and marker_stream are the names of the streams being used
eeg_stream = [s for s in streams if s['info']['name'][0] == 'eeg'][0]
marker_stream = [s for s in streams if s['info']['name'][0] == 'PsychoPyMarkers'][0]

# Retrieve EEG signal data and markers
eeg_data = eeg_stream['time_series']
markers = marker_stream['time_series']

# Define start and end markers for SSVEP
ssvep_5hz_trigger_start = 2  # Start trigger for SSVEP 5 Hz
ssvep_5hz_trigger_end = 12    # End trigger for SSVEP 5 Hz
ssvep_7hz_trigger_start = 4   # Start trigger for SSVEP 7 Hz
ssvep_7hz_trigger_end = 14    # End trigger for SSVEP 7 Hz

# Function to cut segments of the signal
def cut_ssvep_signal(eeg_data, markers, start_trigger, end_trigger, pre_time=1, post_time=2, fs=256):
    segments = []
    for i in range(len(markers) - 1):
        if markers[i][0] == start_trigger and markers[i + 1][0] == end_trigger:
            start = int((markers[i][0] - pre_time) * fs)
            end = int((markers[i + 1][0] + post_time) * fs)
            segment = eeg_data[start:end]  # Ensure segment is (n_samples, n_channels)
            segments.append(segment)
    return np.array(segments)  # Convert to a 3D array (n_epochs, n_samples, n_channels)

# Cut EEG segments for SSVEP 5 Hz and 7 Hz
eeg_segments_5hz = cut_ssvep_signal(eeg_data, markers, ssvep_5hz_trigger_start, ssvep_5hz_trigger_end)
eeg_segments_7hz = cut_ssvep_signal(eeg_data, markers, ssvep_7hz_trigger_start, ssvep_7hz_trigger_end)

# Check shapes of the segments
print(f'Shape of 5 Hz segments: {eeg_segments_5hz.shape}')
print(f'Shape of 7 Hz segments: {eeg_segments_7hz.shape}')

# Create MNE information structure
n_channels = eeg_data.shape[1]  # Number of channels from the main stream
info = mne.create_info(ch_names=[f'EEG_{i+1}' for i in range(n_channels)], sfreq=256, ch_types='eeg')

# Create MNE EpochsArray
epochs_5hz = mne.EpochsArray(eeg_segments_5hz.transpose(0, 2, 1), info)  # Shape: (n_epochs, n_samples, n_channels)
epochs_7hz = mne.EpochsArray(eeg_segments_7hz.transpose(0, 2, 1), info)

# Save to XDF file
def write_xdf(filename, epochs_list, markers):
    # Create the root element
    root = ET.Element('XDF')

    for i, epochs in enumerate(epochs_list):
        data = epochs.get_data()  # Get the data (shape: n_epochs, n_channels, n_samples)
        timestamps = np.arange(data.shape[1]) / epochs.info['sfreq']  # Generate timestamps

        # Create a stream element
        stream = ET.SubElement(root, 'Stream')

        # Add stream information
        info = ET.SubElement(stream, 'Info')
        name = ET.SubElement(info, 'Name')
        name.text = f'eeg_{i + 1}'

        type_ = ET.SubElement(info, 'Type')
        type_.text = 'EEG'

        # Add channels
        channels = ET.SubElement(info, 'Channels')
        for ch in epochs.info['ch_names']:
            channel = ET.SubElement(channels, 'Channel')
            channel.text = ch

        sample_rate = ET.SubElement(info, 'SampleRate')
        sample_rate.text = str(int(epochs.info['sfreq']))

        # Add timestamps
        time_series = ET.SubElement(stream, 'TimeSeries')
        for ts in timestamps:
            timestamp_elem = ET.SubElement(time_series, 'Timestamp')
            timestamp_elem.text = str(ts)

        # Add data
        data_elem = ET.SubElement(stream, 'Data')
        for epoch in data:  # Loop through each epoch
            for channel_data in epoch:  # Loop through each channel
                data_point = ET.SubElement(data_elem, 'DataPoint')
                data_point.text = ','.join(map(str, channel_data))

    # Save markers in a separate stream
    markers_stream = ET.SubElement(root, 'Stream')
    markers_info = ET.SubElement(markers_stream, 'Info')
    markers_name = ET.SubElement(markers_info, 'Name')
    markers_name.text = 'Markers'
    markers_type = ET.SubElement(markers_info, 'Type')
    markers_type.text = 'Markers'

    # Add markers data
    markers_data_elem = ET.SubElement(markers_stream, 'Data')
    for marker in markers:  # Loop through each marker
        marker_elem = ET.SubElement(markers_data_elem, 'Marker')
        marker_elem.text = str(marker[0])

    # Create the tree and write to file
    tree = ET.ElementTree(root)
    tree.write(filename, encoding='utf-8', xml_declaration=True)

# Call the write function
write_xdf('/content/sub-DataSSVEP551_ses-S002_task-Default_run-001_eeg_new.xdf', [epochs_5hz, epochs_7hz], markers)


ModuleNotFoundError: No module named 'mne'

In [ ]:
import pyxdf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft
from scipy.signal import butter, filtfilt
from google.colab import files
import io

# อัปโหลดไฟล์
uploaded = files.upload()

# ตรวจสอบชื่อไฟล์ที่อัปโหลดและอ่านไฟล์
file_name = list(uploaded.keys())[0]  # เลือกไฟล์แรกที่อัปโหลด

# Load data from .xdf file
def load_data(filepath):
    streams, header = pyxdf.load_xdf(filepath)

    eeg_stream = None
    marker_stream = None
    for stream in streams:
        if stream['info']['name'][0] == 'eeg' and stream['info']['type'][0] == 'signal':
            eeg_stream = stream
        elif stream['info']['type'][0] == 'Markers':
            marker_stream = stream

    if eeg_stream is None:
        raise ValueError(f"ไม่พบ stream ชื่อ 'eeg' ในไฟล์ {filepath}")

    data = eeg_stream['time_series']
    sampling_rate = float(eeg_stream['info']['nominal_srate'][0])
    channel_names = eeg_stream['info']['desc'][0]['channels'][0]['channel']
    channel_labels = [chan['label'][0] for chan in channel_names]

    df = pd.DataFrame(data, columns=channel_labels)

    markers = []
    if marker_stream:
        marker_times = marker_stream['time_stamps']
        marker_values = marker_stream['time_series']
        markers = list(zip(marker_times, marker_values))

    return df, sampling_rate, markers

# Bandpass filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Plot Power Spectrum with FFT and multiple target frequencies
def plot_power_spectrum_fft(df, sampling_rate, markers, channels, title_prefix, color='blue', target_freqs=[5, 7], bandwidth=2):
    plt.figure(figsize=(15, 20))

    for i, channel in enumerate(channels):
        if channel in df.columns:
            signal = df[channel].values

            # Apply bandpass filter if needed
            filtered_signal = bandpass_filter(signal, 1, 50, sampling_rate)  # Example: 1-50 Hz filtering

            # FFT of the filtered signal
            N = len(filtered_signal)
            T = 1.0 / sampling_rate
            yf = fft(filtered_signal)
            xf = np.fft.fftfreq(N, T)[:N // 2]
            power_spectrum = 2.0 / N * np.abs(yf[:N // 2])

            # Plot power spectrum
            plt.subplot(len(channels), 1, i + 1)
            plt.plot(xf, power_spectrum, color=color, label=f'{title_prefix} {channel}')
            plt.title(f'Power Spectrum for {channel}')
            plt.xlabel('Frequency (Hz)')
            plt.ylabel('Power')

            # Set frequency range to 0-10 Hz
            plt.xlim(0, 10)

            # Set ticks for frequencies from 1 to 10 Hz
            plt.xticks(np.arange(1, 11, 1))

            # Highlight the target frequency ranges (5 Hz and 7 Hz)
            for target_freq in target_freqs:
                plt.axvspan(target_freq - bandwidth, target_freq + bandwidth, color='orange', alpha=0.3, label=f'Target: {target_freq} Hz ±{bandwidth} Hz')

            # Mark triggers with vertical lines in the frequency domain
            for timestamp, marker in markers:
                plt.axvline(x=timestamp, color='red', linestyle='--', label=f'Trigger {marker[0]}')

            plt.legend()
        else:
            print(f'Warning: {channel} not found in data columns.')

    plt.tight_layout()

# โหลดข้อมูลจากไฟล์ที่อัปโหลด
df1, sampling_rate1, markers1 = load_data(io.BytesIO(uploaded[file_name]))

# แสดง Marker เพื่อการตรวจสอบ
print("Markers from File 1:")
for timestamp, marker in markers1:
    print(f"Timestamp: {timestamp:.2f}, Marker: {marker[0]}")

# เลือกช่องสัญญาณที่ต้องการแสดงผล
channels = ['O1', 'Fz', 'O2', 'Cz', 'Pz']

# แสดงผล Power Spectrum โดยมีความถี่เป้าหมายที่ 5 และ 7 Hz
plot_power_spectrum_fft(df1, sampling_rate1, markers1, channels, 'File 1', color='blue', target_freqs=[5, 7], bandwidth=2)

# แสดงกราฟ
plt.show()


In [ ]:
import pyxdf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft
from scipy.signal import butter, filtfilt
from google.colab import files
import io

# Function to upload and load data from XDF file
def load_data(filepath):
    streams, header = pyxdf.load_xdf(filepath)

    eeg_stream = None
    marker_stream = None
    for stream in streams:
        if stream['info']['name'][0] == 'eeg' and stream['info']['type'][0] == 'signal':
            eeg_stream = stream
        elif stream['info']['type'][0] == 'Markers':
            marker_stream = stream

    if eeg_stream is None:
        raise ValueError(f"ไม่พบ stream ชื่อ 'eeg' ในไฟล์ {filepath}")

    data = eeg_stream['time_series']
    sampling_rate = float(eeg_stream['info']['nominal_srate'][0])
    channel_names = eeg_stream['info']['desc'][0]['channels'][0]['channel']
    channel_labels = [chan['label'][0] for chan in channel_names]

    df = pd.DataFrame(data, columns=channel_labels)

    markers = []
    if marker_stream:
        marker_times = marker_stream['time_stamps']
        marker_values = marker_stream['time_series']
        markers = list(zip(marker_times, marker_values))

    return df, sampling_rate, markers

# Bandpass filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Plot Power Spectrum with FFT and highlight target frequencies
def plot_power_spectrum_fft(df, sampling_rate, markers, channels, title_prefix, color='blue', target_freqs=[5, 7], bandwidth=2):
    plt.figure(figsize=(15, 20))

    for i, channel in enumerate(channels):
        if channel in df.columns:
            signal = df[channel].values

            # Apply bandpass filter
            filtered_signal = bandpass_filter(signal, 1, 50, sampling_rate)  # Example: 1-50 Hz filtering

            # FFT of the filtered signal
            N = len(filtered_signal)
            T = 1.0 / sampling_rate
            yf = fft(filtered_signal)
            xf = np.fft.fftfreq(N, T)[:N // 2]
            power_spectrum = 2.0 / N * np.abs(yf[:N // 2])

            # Plot power spectrum
            plt.subplot(len(channels), 1, i + 1)
            plt.plot(xf, power_spectrum, color=color, label=f'{title_prefix} {channel}')
            plt.title(f'Power Spectrum for {channel}')
            plt.xlabel('Frequency (Hz)')
            plt.ylabel('Power')

            # Set frequency range to 0-10 Hz
            plt.xlim(0, 10)

            # Set ticks for frequencies from 1 to 10 Hz
            plt.xticks(np.arange(1, 11, 1))

            # Highlight the target frequency ranges (5 Hz and 7 Hz)
            for target_freq in target_freqs:
                plt.axvspan(target_freq - bandwidth, target_freq + bandwidth, color='orange', alpha=0.3, label=f'Target: {target_freq} Hz ±{bandwidth} Hz')

            # Mark triggers with vertical lines in the frequency domain
            for timestamp, marker in markers:
                plt.axvline(x=timestamp, color='red', linestyle='--', label=f'Trigger {marker[0]}')

            plt.legend()
        else:
            print(f'Warning: {channel} not found in data columns.')

    plt.tight_layout()

# Upload the file
uploaded = files.upload()

# Check the uploaded file name and read the file
file_name = list(uploaded.keys())[0]  # Select the first uploaded file

# Load data from the uploaded file
df1, sampling_rate1, markers1 = load_data(io.BytesIO(uploaded[file_name]))

# Display markers for verification
print("Markers from File 1:")
for timestamp, marker in markers1:
    print(f"Timestamp: {timestamp:.2f}, Marker: {marker[0]}")

# Select channels to display
channels = ['O1', 'Fz', 'O2', 'Cz', 'Pz']

# Display Power Spectrum highlighting target frequencies at 5 and 7 Hz
plot_power_spectrum_fft(df1, sampling_rate1, markers1, channels, 'File 1', color='blue', target_freqs=[5, 7], bandwidth=2)

# Show the plot
plt.show()
